In [3]:
## 필요라이브러리
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [38]:
# 모든 정거장 위치 검색하기
allBusStop = []
def busStop ():
    url = 'http://apis.data.go.kr/6260000/BusanBIMS/busStopList'
    params ={'serviceKey' : 'l2SER5UdFnVMTCfW/75Op1w1xj0ZVJyWjxZCA4s2qcMkJIjXonsQeorj8k35mmylOV0ywqc75RoXXtagwXuUnQ==',
              'pageNo' : '1',
              'numOfRows' : '499'}

    response = requests.get(url, params=params)
    decodecontent = response.content.decode('utf-8') 
    xmlParser = BeautifulSoup(decodecontent,'xml')
    items= xmlParser.select("item")
    for number in range(0,len(items)):
        item = items[number].text.split('\n')
        item.remove('')
        item.remove('')
        allBusStop.append(item)
busStop()

In [75]:
busnStopDF = pd.read_excel('./data/부산광역시_부산버스정보시스템.xlsx')

In [78]:
condition = busnStopDF['정류장 구분'] == '일반'
stoptGeneral = busnStopDF[condition]
stoptGeneral.to_excel('./data/부산버스_일반.xlsx',index=False)

In [77]:
condition = busnStopDF['정류장 구분'] == '마을'
stopMura = busnStopDF[condition]

In [ ]:
stopMura.to_excel('./data/부산버스_마을.xlsx',index=False)